In [3]:
import csv
import pandas as pd
import os


In [8]:
files

'item_review.2018-09.tsv'

In [11]:
path = "/home/hc/[NII-IDR] 楽天市場データ/review/unzip"
files = os.listdir(path)[0]
file_path = os.path.join(path,files)

col = ['投稿者ID', '店舗名', '店舗ID', '商品名', '商品ID', '商品ページURL', '商品ジャンルID', '商品ジャンルIDパス', '使い道',
                        '目的', '頻度', '評価ポイント', 'レビュータイトル', 'レビュー内容', '参考になった数', 'レビュー登録日時']
data = pd.read_csv(file_path,sep='\t',quoting=csv.QUOTE_NONE,names=col)
data['レビュー内容'].to_csv("0.csv")